In [1]:
import os
os.chdir("../src")
%pwd

'/home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src'

In [2]:
import pandas as pd
from tqdm.auto import tqdm

from pathlib import Path
from russian_news_sentiment_analysis.utils.common import init_spark_session, load_txt
from russian_news_sentiment_analysis.config.configuration import ConfigurationManager

manager = ConfigurationManager()

[2024-03-31 07:16:35,714: 67: logger: INFO: common:  yaml file: russian_news_sentiment_analysis/config/config.yaml loaded successfully]


In [3]:
spark = init_spark_session()

In [4]:
df = spark.read.parquet(os.path.join(manager.config.path.raw_data, 'linis-crowd-2015.parquet')).toPandas()

In [7]:
df['text'].tolist()[:20]

['при частотности ложных сигналов тревоги не более 1/24.В настоящее время программа разработки системы находится на 7-м уровне технологической готовности (Technology Readiness Level), что означает проведение испытаний опытного образца в боевых условиях.По заявлению пресс-секретаря компании Элбит сиситемз, работы по модернизации, направленные на обеспечение перекрытия секторов обзора сенсоров устанавливаемой на ББМ системы оповещения об облучении ELWS, позволили повысить точность наведения по азимуту боевого модуля на угрозу до 1 град.Представители Элбит системз также заявили о разработке системы активной защиты ББМ нового поколения класса soft-kill, однако отказались предоставить какую-либо уточняющую информацию о ней. Данное оборудование планируется представить в ходе международной выставки продукции военного назначения DSEi (10.09.2013 13.09.2013) в Лондоне.Материалы по теме:Противотанковые ракетные комплексы Тульского КБПБоевая машина многоцелевого ракетного комплекса Корнет-ЭМ',
 '

In [8]:
df = spark.read.parquet(os.path.join(manager.config.path.raw_data, 'rusentiment.parquet')).toPandas()

In [9]:
df['text'].tolist()[:20]

['К Нам Приходит Праздник....',
 'жду своего котика',
 'опять чемоданы',
 'девченки,жду!!!',
 'кусочек :) даже Таранухина :)',
 'Где купить аппарат Ulthera System? И сколько стоит?',
 '-Вова, ты остаешься в Томске\n- ........',
 'Ахахаха;Dлюбимая сеструха *___*',
 'ЕБУЧИЕ ШЕЙХИ ПОШЛИ НАХУЙ',
 'Давай людям второй шанс ! Нам свойственно ошибаться ты сама это знаешь. Да и как не крути люди меняются =)',
 'Пытаться забыть человека равносильно тому, чтобы вспомнить человека, которого ты никогда не видел.',
 'мне фоновая музыка очень нравится)',
 'Просто нет слов))',
 'АНСАМБЛЬ ХРИСТА СПАСИТЕЛЯ И МАТЬ СЫРА ЗЕМЛЯ   Наши иконы rip',
 'манеры - способ уважать др. людей',
 'Игнорируете:(',
 'гитлер знал о НЛО и ПОЛОЙ ЗЕМЛЕ  (по следам гитлера)',
 'ну что когда уже приедешь?)',
 'Ничего такая банька:)))',
 'У Меня Самая Красивая,и самая Лучшая Жена на Планете Земля!!!!!']

In [4]:
spark = init_spark_session()

load_txt(Path(manager.config.path.hadoop_files_checklist))
files = [k.split('/')[-1] for k in load_txt(Path(manager.config.path.hadoop_files_checklist)) if 'prepared/' in k]
datasets = [k.split(".")[0] for k in files]

quantiles = [round(0.1*k, 1) for k in range(1, 10)]
statistics = pd.DataFrame(columns=['dataset', 'size', '0_cnt', '1_cnt', 'min', 'max', 'mean'] + [f'q_{int(k*100)}' for k in quantiles])

for (dataset, file) in tqdm(zip(datasets, files), total=len(files)):
    df = spark.read.parquet(os.path.join(manager.config.path.raw_data, file)).toPandas()
    df['sizes'] = df['text'].apply(lambda x: len(x.split(' ')))
    qs = [df['sizes'].quantile(k) for k in quantiles]
    row = [dataset, len(df), round(df["label"].value_counts()[0]/len(df)*100), round(df["label"].value_counts()[1]/len(df)*100), 
           df['sizes'].min(), df['sizes'].max(), df['sizes'].mean() ] + qs
    statistics.loc[len(statistics)] = row
    
statistics = statistics.set_index('dataset')

spark.stop()

[2024-03-29 07:04:12,452: 167: logger: INFO: common:  txt file loaded successfully from: /home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src/logs/hadoop_files.txt]
[2024-03-29 07:04:12,454: 167: logger: INFO: common:  txt file loaded successfully from: /home/jovyan/work/sentiment_analysis_in_russian_news_feeds/src/logs/hadoop_files.txt]


  0%|          | 0/8 [00:00<?, ?it/s]

In [4]:
statistics

,size,0_cnt,1_cnt,min,max,mean,q_10,q_20,q_30,q_40,q_50,q_60,q_70,q_80,q_90
dataset,,,,,,,,,,,,,,,
kaggle-competitions-sentiment-analysis-in-russian,7949,17,83,4,49039,505.438923,92.8,145.6,183.0,224.0,269.0,331.0,415.0,571.0,987.6
kinopoisk,36579,13,87,6,2054,218.533093,76.0,105.0,130.0,154.0,182.0,214.0,254.0,313.0,411.0
linis-crowd-2015,20378,37,63,1,3284,158.165374,84.0,103.0,118.0,131.0,144.0,158.0,174.0,197.0,235.0
linis-crowd-2016,15346,40,60,3,1658,158.335918,85.0,103.0,117.0,130.0,144.0,159.0,177.0,201.0,243.0
ru-tweet-corp,217440,49,51,1,94,12.102410,7.0,8.0,9.0,10.0,11.0,12.0,14.0,16.0,19.0
rureviews,87127,33,67,1,196,19.538892,5.0,7.0,9.0,11.0,14.0,17.0,22.0,28.0,41.0
rusentiment,21019,17,83,1,200,12.186070,2.0,3.0,4.0,5.0,7.0,9.0,12.0,16.0,26.0
russian-news-titles-test,51121,34,66,1,66,6.328554,4.0,5.0,5.0,6.0,6.0,7.0,7.0,7.0,8.0


In [11]:
cols = ['q_10', 'q_30', 'q_50', 'q_70', 'q_90', 'max']
for i in statistics.index:
    print(f'{i}: ', ' / '.join([str(round(statistics.loc[i, col])) for col in cols]))

kaggle-competitions-sentiment-analysis-in-russian:  93 / 183 / 269 / 415 / 988 / 49039
kinopoisk:  76 / 130 / 182 / 254 / 411 / 2054
linis-crowd-2015:  84 / 118 / 144 / 174 / 235 / 3284
linis-crowd-2016:  85 / 117 / 144 / 177 / 243 / 1658
ru-tweet-corp:  7 / 9 / 11 / 14 / 19 / 94
rureviews:  5 / 9 / 14 / 22 / 41 / 196
rusentiment:  2 / 4 / 7 / 12 / 26 / 200
russian-news-titles-test:  4 / 5 / 6 / 7 / 8 / 66
